In [1]:
from pyuvdata import UVData, UVCal, utils
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import time
from glob import glob
import os
import ja

In [2]:
# Get the files which satisfy the sun cut
imagepath = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2458098/'
rawpath = '/lustre/aoc/projects/hera/H1C_IDR2/2458098/'
calpath = '/lustre/aoc/projects/hera/jaguirre/PolarizedMosaics/Test/'
imagefits = glob(imagepath+'zen.*.HH.calibrated.uvh5_image/*.image.image.fits')
uvh5stem = []
uvh5files = []
for i,imf in enumerate(imagefits):
    tmp = os.path.split(imf)[-1].split('.')
    uvh5stem.append(tmp[0]+'.'+tmp[1]+'.'+tmp[2])
    uvh5files.append(calpath+uvh5stem[i]+'.HH.calibrated.uvh5')
uvh5stem.sort()
uvh5files.sort()
Nfiles = len(uvh5files)

In [3]:
timer = ja.Timer()
timer2 = ja.Timer()

In [4]:
# Build a place to hold the running average by baseline
uvd = UVData()
uvd.read(uvh5files[0])

In [5]:
bl_avg = {}
bl_hits = {}
for key, data in uvd.antpairpol_iter():
    bl_avg[key] = np.zeros([Nfiles,1024],dtype='complex128')
    bl_hits[key] = np.zeros([Nfiles,1024],dtype='float128')

In [6]:
print(len(bl_avg))
print(len(bl_hits))
print(bl_avg[key].shape)
print(bl_hits[key].shape)
print(uvd.Nbls*4)

5512
5512
(56, 1024)
(56, 1024)
5512


In [ ]:
# Can we dump this somewhere via hdf5?


In [ ]:
timer.start()
# Extract waterfalls baseline-by-baseline
for i,f in enumerate(uvh5files):
    print(f)
    timer2.start()
    uvd = UVData()
    uvd.read(f)
    timer2.stop('Reading data')

    timer2.start()
    for key, data in uvd.antpairpol_iter():
        # do something with data to make new_data that's the same size
        blt1, blt2, pol = uvd._key2inds(key)
        assert len(blt2) == 0  # should be true for H1C data
        # Grab the data corresponding to that baseline
        wf = uvd.data_array[blt1, 0, :, pol[0]]
        wfflags = ~uvd.flag_array[blt1, 0, :, pol[0]]
        hits = wfflags.sum(axis = 0)
        wfavg = (wf * (wfflags)).sum(axis = 0) / hits
        bl_avg[key][i,:] = wfavg # Accumulate the sum
        bl_hits[key][i,:] = hits
    timer2.stop('Building average')
        
timer.stop('Average calculation')
np.savez('2458098_avg.npz',bl_avg=bl_avg, bl_hits=bl_hits)

In [ ]:
np.savez('2458098_avg.npz',bl_avg=bl_avg)

In [ ]:
np.savez('2458098_avg.npz',bl_avg=bl_avg)

In [ ]:
np.savez('2458098_hits.npz',bl_hits=bl_hits)

In [ ]:
bl_hits[(25,26,'xx')].shape

In [ ]:
len(bl_hits)

In [ ]:
plt.plot(bl_avg[(26,27,'xx')].mean(axis=0).real)
plt.plot(bl_avg[(26,27,'xx')].mean(axis=0).imag)

In [ ]:
plt.plot(bl_avg[(82,143,'xx')].mean(axis=0).real)
plt.plot(bl_avg[(82,143,'xx')].mean(axis=0).imag)